## Broken Power Law Examples

Below are a few examples of the broken power law rating curve implementation.

In [ ]:
from numpy.typing import ArrayLike
import numpy as np
import matplotlib.pyplot as plt
import arviz as az

Equation for a broken power law with $n$ breaks is:

\begin{equation}
f(x) = \begin{cases}
            Ax^{\alpha_0} & {\rm if}\ x \leq x_1 \\
            Ax^{\alpha_1}x_1^{\alpha_0 - \alpha_1} & {\rm if}\ x_1 < x \leq x_2 \\
            ... \\
            Ax^{\alpha_n} {\displaystyle \prod^{n}_{i=1}} x_i^{\alpha_{i-1} - \alpha_{i}} & {\rm if}\ x > x_n
       \end{cases},
\end{equation}

where $A$ is the scaling amplitude, $\alpha_0$ is the power law slope before the first breakpoint ($x_1$), $\alpha_i$ are the power law slopes between breakpoints $x_{i}$ and $x_{i+1}$ with $i$ being the $i$th breakpoint and $i < n$, and $\alpha_n$ is the power law slope after the last breakpoint ($x_n$).

In log-space this translates to:

\begin{equation}
\log(f(x)) = \begin{cases}
            \log(A) + \alpha_0 \log(x) & {\rm if}\ x \leq x_1 \\
            \log(A) + \alpha_1 \log(x) + (\alpha_0 - \alpha_1) \log(x_1) & {\rm if}\ x_1 < x \leq x_2 \\
            ... \\
            \log(A) + \alpha_n \log(x) + {\displaystyle \sum^{n}_{i=1}} (\alpha_{i-1} - \alpha_{i}) \log(x_i) & {\rm if}\ x > x_n
       \end{cases}.
\end{equation}

With $x_0$ offset:

\begin{equation}
\log(f(x)) = \begin{cases}
            \log(A) + \alpha_0 \log(x - x_0) & {\rm if}\ x - x_0 \leq x_1 \\
            \log(A) + \alpha_1 \log(x - x_0) + (\alpha_0 - \alpha_1) \log(x_1) & {\rm if}\ x_1 < x - x_0 \leq x_2 \\
            ... \\
            \log(A) + \alpha_n \log(x - x_0) + {\displaystyle \sum^{n}_{i=1}} (\alpha_{i-1} - \alpha_{i}) \log(x_i) & {\rm if}\ x - x_0 > x_n
       \end{cases}.
\end{equation}

In [ ]:
def BrokenPowerLaw(x: ArrayLike, amplitude: float, breaks: ArrayLike, alphas: ArrayLike) -> np.ndarray:
    x = np.array(x)
    alphas_array = np.array(alphas)
    breaks = np.array(breaks)
    cumprod = np.cumprod((breaks[1:]) ** (alphas_array[:-1] - alphas_array[1:]))
    # Product for first element is 1, as it does not have a product
    cumprod = np.insert(cumprod, 0, 1).reshape((-1, 1))
    # Create ranges for each segment
    segments_range = np.insert((breaks[1:]), [0, len(breaks[1:])], [0.0, np.inf]).reshape((-1, 1))

    # Arrays are broadcasts for vectorized computation. Calculates function within range sets value to 0 everywhere else. 
    #   Then sum along segment dimension to collapse.
    fx = np.where(((x- breaks[0]) > segments_range[:-1]) & ((x- breaks[0]) <= segments_range[1:]), 
                  amplitude * (x- breaks[0]) ** alphas_array.reshape((-1, 1)) * cumprod, 0)
    fx = fx.sum(axis=0)
    
    return fx

Now that we have created an equation function, let's generate some simulated data.

In [ ]:
h = np.geomspace(3, 20, 201)
a = 0.01
breaks = [0, 7.0]
alphas = [0.9, 2]
q = BrokenPowerLaw(h, a, breaks, alphas)
# Add some random noise in log space
q = np.exp(np.log(q) + np.random.randn(len(q)) * 0.01)

fig, ax = plt.subplots()
ax.loglog(q, h, marker='o', linestyle='')
ax.set(xlabel='Discharge', ylabel='Stage')
plt.show

Then fit the simulated data and plot the fits.

In [ ]:
from ratingcurve.experimental_ratings import BrokenPowerLawRating

# initialize the model
# Default model configurations are okay, as we want uniform priors and 2 segments
brokenpowerrating = BrokenPowerLawRating()
brokenpowerrating.fit(h, q)
data = brokenpowerrating.table()

In [ ]:
fig, ax = plt.subplots()
ax.loglog(q, h, marker='o', linestyle='', label='Data')
ax.loglog(data['discharge'], data['stage'], label='Model Fit')
ax.set(xlabel='Discharge', ylabel='Stage')
plt.legend()
plt.show

In [ ]:
brokenpowerrating.plot()

In [ ]:
brokenpowerrating.plot_residuals()

In [ ]:
print('Break point expected values: '+str(breaks)+', Derived values: '
      +str(brokenpowerrating.idata.posterior.hs.mean(dim=['draw']).values.squeeze())+' +/- '+str(brokenpowerrating.idata.posterior.hs.std(dim=['draw']).values.squeeze()))

Now let's test the model on some real data to see how it compares on something not generated by itself.

In [ ]:
from ratingcurve import data

# load tutorial data
df = data.load('green channel')

fig, ax = plt.subplots()
ax.errorbar(df['q'], df['stage'], xerr=df['q_sigma'], marker='o', linestyle='', fillstyle='none')
ax.set(xlabel='Discharge', ylabel='Stage', xscale='log', yscale='log')
plt.show

In [ ]:
# initialize the model
# Default model config is still good here
brokenpowerrating_gc = BrokenPowerLawRating()
brokenpowerrating_gc.fit(df['stage'], df['q'], df['q_sigma'], method='nuts')
data_gc = brokenpowerrating_gc.table()

Time to plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(df['q'], df['stage'], xerr=df['q_sigma'], marker='o', linestyle='', fillstyle='none', label='Data')
ax.loglog(data_gc['discharge'], data_gc['stage'], label='Smooth Model Fit')
ax.set(xlabel='Discharge', ylabel='Stage', xscale='log', yscale='log')
plt.legend()
plt.show

In [ ]:
brokenpowerrating_gc.summary(var_names=['hs'])

In [ ]:
brokenpowerrating_gc.plot()